# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saipan,15.1355,145.7010,85.05,73,100,7.96,MP,1723510886
1,1,vilyuchinsk,52.9306,158.4028,60.78,89,100,0.38,RU,1723510887
2,2,isafjordur,66.0755,-23.1240,43.59,96,100,18.90,IS,1723510888
3,3,kapuskasing,49.4169,-82.4331,75.29,64,40,3.44,CA,1723510889
4,4,huarmey,-10.0681,-78.1522,62.60,80,14,8.52,PE,1723510890


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
map_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 650,
    frame_height = 800,
    size = "Humidity",
    color = "City"
)

# Display the map
map_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
weather = city_data_df.loc[(city_data_df["Max Temp"] < 80)
                           & (city_data_df["Max Temp"] > 70)
                           & (city_data_df["Wind Speed"] < 8)
                           & (city_data_df["Cloudiness"] < 30)
                           & (city_data_df["Humidity"] < 60),:].dropna()

# Display sample data
weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,santa luzia,-19.7697,-43.8514,70.88,18,0,5.75,BR,1723510928
85,85,birjand,32.8663,59.2211,70.27,23,0,6.91,IR,1723510985
95,95,baiyin,36.5583,104.2081,72.95,39,12,3.65,CN,1723510996
100,100,st. paul,44.9444,-93.0933,77.47,51,0,3.44,US,1723510827
115,115,pingliang,35.5392,106.6861,73.06,56,2,4.54,CN,1723511019
119,119,barro alto,-11.7608,-41.9117,74.05,52,0,3.58,BR,1723511023
202,202,senta,45.9275,20.0772,76.01,48,0,7.47,RS,1723511120
288,288,karratha,-20.7377,116.8463,75.92,38,16,5.55,AU,1723511219
385,385,flin flon,54.7682,-101.8650,77.97,32,0,4.94,CA,1723511329
395,395,genhe,50.7833,121.5167,77.49,44,0,1.68,CN,1723511340


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
35,santa luzia,BR,-19.7697,-43.8514,18,
85,birjand,IR,32.8663,59.2211,23,
95,baiyin,CN,36.5583,104.2081,39,
100,st. paul,US,44.9444,-93.0933,51,
115,pingliang,CN,35.5392,106.6861,56,
119,barro alto,BR,-11.7608,-41.9117,52,
202,senta,RS,45.9275,20.0772,48,
288,karratha,AU,-20.7377,116.8463,38,
385,flin flon,CA,54.7682,-101.8650,32,
395,genhe,CN,50.7833,121.5167,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
            "limit": 20,
            "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
santa luzia - nearest hotel: Alcoboca
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
baiyin - nearest hotel: 红鹭宾馆
st. paul - nearest hotel: Crowne Plaza
pingliang - nearest hotel: 平凉新都酒店
barro alto - nearest hotel: No hotel found
senta - nearest hotel: Hotel Royal
karratha - nearest hotel: Karratha International Hotel
flin flon - nearest hotel: Copperbelt Hotel
genhe - nearest hotel: No hotel found
the pas - nearest hotel: Wescana Inn
menomonie - nearest hotel: Cobblestone Inn & Suites
choyr - nearest hotel: Сансар Хайрхан зочид буудал
chillicothe - nearest hotel: Quality Inn Chillicothe
cobija - nearest hotel: Hospedaje Novedades
buka - nearest hotel: No hotel found
guajara mirim - nearest hotel: Novo Hotel Campos
sinj - nearest hotel: Hotel Alkar
bukama - nearest hotel: Mission catholique
dauphin - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
35,santa luzia,BR,-19.7697,-43.8514,18,Alcoboca
85,birjand,IR,32.8663,59.2211,23,مهمانسرای جهانگردی بیرجند
95,baiyin,CN,36.5583,104.2081,39,红鹭宾馆
100,st. paul,US,44.9444,-93.0933,51,Crowne Plaza
115,pingliang,CN,35.5392,106.6861,56,平凉新都酒店
119,barro alto,BR,-11.7608,-41.9117,52,No hotel found
202,senta,RS,45.9275,20.0772,48,Hotel Royal
288,karratha,AU,-20.7377,116.8463,38,Karratha International Hotel
385,flin flon,CA,54.7682,-101.8650,32,Copperbelt Hotel
395,genhe,CN,50.7833,121.5167,44,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 650,
    frame_height = 650,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)